##Imports

In [1]:
import re
import pandas as pd
import json
import numpy as np
import seaborn as sns
import nltk
from bertopic import BERTopic

/home/lucas/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Extração de Dados

In [2]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df


def df_tweets_respostas(json_filename):
    df = pd.read_json(json_filename).dropna().reset_index()[['replies']]
    ds = []
    for replies in df['replies'].to_list():
        for reply in replies:
            ds.append(reply)

    reply_df = pd.DataFrame (ds).drop(columns = ['attachments'])
    return reply_df

def df_tweets_cadidatos_respostas(json_filename):
    df_tweets_cand = df_tweets_candidatos(json_filename)
    df_tweets_reps = df_tweets_respostas(json_filename)

    return (df_tweets_cand, df_tweets_reps)

# Pré-Processamento

In [3]:
tweets = df_tweets_candidatos('./datasets/dataset.json')


# Adicionar nomes dos candidatos no dataframe
for index, row in tweets.iterrows():
    candidato = ''
    if row.author_id == 2670726740:
      candidato = 'lula'
    elif row.author_id == 128372940:
      candidato = 'bolsonaro'
    elif row.author_id == 33374761:
      candidato = 'ciro'
    else:
      candidato = 'n/d'
    tweets.at[index, 'candidato'] = candidato

tweets = tweets.sample(frac=1)
print(f'Número de tweets: {len(tweets)}\n')
print(tweets.head())

Número de tweets: 300

      author_id      conversation_id                created_at  \
102    33374761  1560396434725376000 2022-08-18 22:41:15+00:00   
60   2670726740  1559313079334322176 2022-08-15 22:56:23+00:00   
70   2670726740  1558948211250937856 2022-08-14 22:46:32+00:00   
241   128372940  1558491746102906880 2022-08-13 16:32:42+00:00   
256   128372940  1557765109094797312 2022-08-11 16:32:40+00:00   

                                                  text                   id  \
102  Pare pra pensar e me responda: se Bolsonaro ou...  1560396434725376000   
60   Nós nordestinos, quando a gente se perde, a ge...  1559313079334322176   
70   Bolsonaro tenta se aproveitar do PIX, mas na r...  1558948211250937856   
241  - Agradeço aos mais de 450 mil espectadores qu...  1558491746102906880   
256  - A redução representa queda de R$ 0,22 por li...  1557766961546788864   

     candidato  
102       ciro  
60        lula  
70        lula  
241  bolsonaro  
256  bolsonaro  


In [4]:
nltk.download('stopwords')
pt_stop = set(nltk.corpus.stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import re
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def preprocess_text(document):

        # removing urls
        document = re.sub(r'http\S+', '', document)

        # removing mentions
        document = re.sub(r'\B@\w+', '', document)

        # remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # converting to lowercase
        document = document.lower()

        # lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in pt_stop]
        tokens = [word for word in tokens if len(word) > 3]

        return ' '.join(tokens)

# Modelagem de Tópicos

In [6]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /home/lucas/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lucas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# create model 
model = BERTopic(language="multilingual", verbose=True, min_topic_size=3, top_n_words=10, calculate_probabilities=True)

# convert to list 
docs = tweets.text.to_list()

# Pre-processamento
for i, doc in enumerate(docs):
    docs[i] = preprocess_text(doc)

tweets_topic, probabilities = model.fit_transform(docs)

Batches: 100%|██████████| 10/10 [00:05<00:00,  1.91it/s]
2022-08-25 18:07:05,908 - BERTopic - Transformed documents to Embeddings
2022-08-25 18:07:14,398 - BERTopic - Reduced dimensionality
2022-08-25 18:07:14,456 - BERTopic - Clustered reduced embeddings


Adicionando o tópico de cada tweet no Dataframe

In [8]:
topic_names = model.generate_topic_labels(nr_words=4)

# Array indicando o tópico de cada tweet (numérico -> índice do array "topic_names" + 1)
# print(tweets_topic)
# Array com todos os tópicos nomeados
# print(topic_names)

for index, tweet_topic in enumerate(tweets_topic):
    tweets.at[index + 1, 'topico'] = topic_names[tweet_topic + 1]

tweets.head()

,author_id,conversation_id,created_at,text,id,candidato,topico
102,3.337476e+07,1.560396e+18,2022-08-18 22:41:15+00:00,Pare pra pensar e me responda: se Bolsonaro ou...,1.560396e+18,ciro,6_pequenos_custou_quanto_básico
60,2.670727e+09,1.559313e+18,2022-08-15 22:56:23+00:00,"Nós nordestinos, quando a gente se perde, a ge...",1.559313e+18,lula,6_pequenos_custou_quanto_básico
70,2.670727e+09,1.558948e+18,2022-08-14 22:46:32+00:00,"Bolsonaro tenta se aproveitar do PIX, mas na r...",1.558948e+18,lula,-1_brasil_campanha_quero_horas
241,1.283729e+08,1.558492e+18,2022-08-13 16:32:42+00:00,- Agradeço aos mais de 450 mil espectadores qu...,1.558492e+18,bolsonaro,6_pequenos_custou_quanto_básico
256,1.283729e+08,1.557765e+18,2022-08-11 16:32:40+00:00,"- A redução representa queda de R$ 0,22 por li...",1.557767e+18,bolsonaro,0_ciro_vivo_lula_equipelula


Abaixo, os tópicos identificados entre todos os tweets

In [9]:
freq = model.get_topic_info()
print(freq)

    Topic  Count                                               Name
0      -1     53                     -1_brasil_campanha_quero_horas
1       0     26                        0_ciro_vivo_lula_equipelula
2       1     25                       1_luladay_noite_gettr_baixar
3       2     21          2_melhor_brasildaesperança_esperança_pais
4       3     16                        3_deus_cristãos_crime_irmão
5       4     15                              4_2022_jair_aula_giro
6       5     15                5_famílias_safra_desemprego_milhões
7       6     13                    6_pequenos_custou_quanto_básico
8       7     13               7_redução_combustíveis_energia_preço
9       8     12                8_ciro_cironorodaviva_pergunta_rota
10      9     11                     9_carta_drogas_saudades_aborto
11     10      9                         10_eleição_voto_falar_café
12     11      8                11_política_corrupção_porque_errado
13     12      8                12_programa_rend

# Análise de Sentimentos

# Similaridade de Textos

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
def monta_docs_tweets(df, topico, coluna_alvo, candidatos):
    lista_tweets = []
    #df_topico = df[df['topico'] == topico]
    df_topico = df

    for cand in candidatos:
        df_cand = df_topico[df_topico['candidato'] == cand]
        lista_tweets_cand = df_cand[coluna_alvo].to_list()
        lista_tweets.append(lista_tweets_cand)
    
    docs = np.concatenate(lista_tweets)
    return docs

def monta_docs_concat_topico(df, topico, coluna_alvo, candidatos):
    docs = []
    df_topico = df[(df['topico'] == topico)]
    for cand in candidatos:
        df_topico_cand = df_topico[df_topico['candidato'] == cand]
        texto = ' '.join(df_topico_cand[coluna_alvo].to_list())
        docs.append(texto)
    return docs


def monta_docs_concat_candidato(df, coluna_alvo, candidatos):
    docs = []
    for cand in candidatos:
        df_cand = df[df['candidato'] == cand]
        texto = ' '.join(df_cand[coluna_alvo].to_list())
        docs.append(texto)
    return docs

stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')

def normaliza_tweet(tweet):
    tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet, re.I|re.A)
    tweet = tweet.lower()
    tweet = tweet.strip()
    tokens = nltk.word_tokenize(tweet)
    tokens_filtrados = [token for token in tokens if token not in stop_words_ptbr]
    tweet_tokens = ' '.join(tokens_filtrados)
    return tweet_tokens

def vetoriza_docs(lista_docs):
    corpus_vetorizado = np.vectorize(lista_docs)
    return corpus_vetorizado

def gera_matriz_tfidf(corpus_vetorizado):
    tf = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
    matriz_tfidf = tf.fit_transform(corpus_vetorizado)
    return matriz_tfidf

def similaridade_pares(matriz_tfidf):
    dict_docs_sim = cosine_similarity(matriz_tfidf)
    df_sim = pd.DataFrame(dict_docs_sim)
    return df_sim

def renomear_colunas(df, candidatos):
    df.columns = candidatos
    return df

def calcula_similaridade_topico(df, topico, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat_topico(df, topico, coluna_alvo, candidatos)
    normaliza_tweets = np.vectorize(normaliza_tweet)
    corpus_normalizado = normaliza_tweets(lista_docs)
    matriz_tfidf = gera_matriz_tfidf(corpus_normalizado)
    df_sim = similaridade_pares(matriz_tfidf)
    df_sim_cands = renomear_colunas(df_sim, candidatos)

    return df_sim_cands

def calcula_similaridade_candidatos(df, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat_candidato(df, coluna_alvo, candidatos)
    normaliza_tweets = np.vectorize(normaliza_tweet)
    corpus_normalizado = normaliza_tweets(lista_docs)
    matriz_tfidf = gera_matriz_tfidf(corpus_normalizado)
    df_sim = similaridade_pares(matriz_tfidf)
    df_sim_cands = renomear_colunas(df_sim, candidatos)

    return df_sim

def calcula_similaridade_topicos(df, topicos, coluna_alvo, candidatos):
    dfs_sim = []
    for topico in topicos:
        dfs_sim.append(calcula_similaridade_topico(df, topico, coluna_alvo, candidatos))
        
    sim_dict = dict(zip(topicos, dfs_sim))
    return sim_dict




In [12]:
df_freq = model.get_topic_info()
topicos_mais_frequentes = df_freq['Name'].to_list()[1:4]
topicos_mais_frequentes

['0_ciro_vivo_lula_equipelula',
 '1_luladay_noite_gettr_baixar',
 '2_melhor_brasildaesperança_esperança_pais']

In [13]:
candidatos = ["lula", "bolsonaro", "ciro"]
analise_similaridade = calcula_similaridade_topicos(tweets, topicos_mais_frequentes, "text", candidatos)
for topico in analise_similaridade.keys():
    print(f"Para tópico = {topico}")
    print(analise_similaridade[topico])
    print('----------------------------------')

Para tópico = 0_ciro_vivo_lula_equipelula
       lula  bolsonaro      ciro
0  1.000000   0.019666  0.030552
1  0.019666   1.000000  0.010793
2  0.030552   0.010793  1.000000
----------------------------------
Para tópico = 1_luladay_noite_gettr_baixar
       lula  bolsonaro      ciro
0  1.000000   0.026899  0.085612
1  0.026899   1.000000  0.027070
2  0.085612   0.027070  1.000000
----------------------------------
Para tópico = 2_melhor_brasildaesperança_esperança_pais
       lula  bolsonaro      ciro
0  1.000000   0.020207  0.053028
1  0.020207   1.000000  0.025134
2  0.053028   0.025134  1.000000
----------------------------------


In [14]:
analise_sim_cand = calcula_similaridade_candidatos(tweets, "text", candidatos)
analise_sim_cand

,lula,bolsonaro,ciro
0,1.000000,0.336546,0.370636
1,0.336546,1.000000,0.307959
2,0.370636,0.307959,1.000000
